In [1]:
import pandas as pd
# from datasets import load_dataset, load_metric

import numpy as np
from datetime import datetime
import torch
# import os
from torch.utils.data import DataLoader
from datasets import Dataset
from tqdm import tqdm
# Importing the T5 modules from huggingface/transformers
from transformers import T5Tokenizer, T5ForConditionalGeneration
from evaluate import load
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer#,DataCollatorForSeq2Seq, 
from torch import cuda
import pickle


In [158]:
# X_test_all_papers_df = pickle.load(open('test_sentences_326papers_45569.pkl','rb'))

In [159]:
# X_test_all_papers_df

,pii,sentence,is_composition
0,S0167273804001882,Electrical and electrochemical properties of a...,0
1,S0167273804001882,Investigations on electrical and electrochemic...,0
2,S0167273804001882,A “quenched [0.75AgI: 0.25AgCl] mixed system/s...,0
3,S0167273804001882,Compositional variation of room temperature co...,0
4,S0167273804001882,The composition: 0.7[0.75AgI: 0.25AgCl]: 0.3[A...,0
...,...,...,...
45564,S0167273814000617,In glass containing more iron oxide the nanost...,0
45565,S0167273814000617,Nanocrystallities are randomly distributed in ...,0
45566,S0167273814000617,Our electrical measurements are consistent wit...,0
45567,S0167273814000617,Two distinct activation energy values in glass...,0


In [4]:
# X_test_all_papers_df_grouped = X_test_all_papers_df.groupby('pii').agg(list).reset_index()

In [5]:
# import re
# def regex_pattern_1(text):
#     # pattern_x = r'x\s*=\s*([\d.]+(?:[,;\s*\d.\s*and]+)*)'
#     pattern_x = r'x\s*=\s*([\d.]+(?:[,;\s*and-–]*[\d.]+)*\b)'
#     # pattern_y = r'y\s*=\s*([\d.]+(?:[,;\s*\d.\s*and]+)*)'
#     pattern_y = r'y\s*=\s*([\d.]+(?:[,;\s*and-–]*[\d.]+)*\b)'
#     # pattern_z_old = r'z\s*=\s*([\d.]+(?:[,;\s*and]*[\d.]+)*)'
#     pattern_z = r'z\s*=\s*([\d.]+(?:[,;\s*and-–]*[\d.]+)*\b)'
    
#     # Extract all the x values from the text
#     x_values = re.findall(pattern_x, text, re.IGNORECASE)
#     y_values = re.findall(pattern_y, text, re.IGNORECASE)
#     z_values = re.findall(pattern_z, text, re.IGNORECASE)
    
#     # print(x_values)
    
#     # print(x_values)
#     result_list_x = []
#     result_list_y = []
#     result_list_z = []
    
#     res = []
    
#     if x_values:
#         for x_val in x_values:
#             x_value = x_val.strip().replace('–', ',')
#             x_value = x_value.strip().replace(';', ',')
#             x_value = x_value.strip().replace('and', ',')
#             values_list = x_value.split(',')
#             result_list_x_t = [value.strip() for value in values_list]

#             temp = []
#             for i in result_list_x_t:
#                 if i.strip() and i.replace('.', '', 1).isdigit():
#                     temp.append(i)
#             result_list_x_t = temp
#             result_list_x.extend(result_list_x_t)
            
        
#     res.append(result_list_x)
    
    
#     if y_values:
#         for y_val in y_values:
#             y_value = y_val.strip().replace('–', ',')
#             y_value = y_value.strip().replace(';', ',')
#             y_value = y_value.strip().replace('and', ',')
#             values_list = y_value.split(',')
#             result_list_y_t = [value.strip() for value in values_list]

#             temp = []
#             for i in result_list_y_t:
#                 if i.strip() and i.replace('.', '', 1).isdigit():
#                     temp.append(i)
#             result_list_y_t = temp
#             result_list_y.extend(result_list_y_t)
            
        
#     res.append(result_list_y)  
    
    
#     if z_values:
#         for z_val in z_values:
#             z_value = z_val.strip().replace('–', ',')
#             z_value = z_value.strip().replace(';', ',')
#             z_value = z_value.strip().replace('and', ',')
#             values_list = z_value.split(',')
#             result_list_z_t = [value.strip() for value in values_list]

#             temp = []
#             for i in result_list_z_t:
#                 if i.strip() and i.replace('.', '', 1).isdigit():
#                     temp.append(i)
#             result_list_z_t = temp
#             result_list_z.extend(result_list_z_t)
            
        
#     res.append(result_list_z)
#     return res
    

In [6]:
# rpattern_values = dict()

# def getXYZ(pii, text_lst):
#     setx = set()
#     sety = set()
#     setz = set()
#     for text in text_lst:
#         rp1 = regex_pattern_1(text)
#         rp2 = [[], [], []]

#         # print(rp1)
#         x_lst = rp1[0]
#         y_lst = rp1[1]
#         z_lst = rp1[2]

#         x_new_lst = [x for x in x_lst if(float(x)<100 and float(x)>0)]
#         y_new_lst = [y for y in y_lst if(float(y)<100 and float(y)>0)]
#         z_new_lst = [z for z in z_lst if(float(z)<100 and float(z)>0)]
    
#         rp1 = [x_new_lst, y_new_lst, z_new_lst]
#         # print(rp1)

#         setx.update(set(rp1[0]+rp2[0]))
#         sety.update(set(rp1[1]+rp2[1]))
#         setz.update(set(rp1[2]+rp2[2]))
        
#     rpattern_values[pii] = [list(setx), list(sety), list(setz)]
#     return rpattern_values[pii]

In [7]:
# X_test_all_papers_df_grouped['xyz'] = X_test_all_papers_df_grouped.apply(lambda row: getXYZ(row['pii'], row['sentence']), axis=1)

In [8]:
# pickle.dump(rpattern_values, open('rpattern_values_test_data_326_papers.pkl', 'wb'))

In [9]:
rpattern_values = pickle.load(open('rpattern_values_test_data_326_papers.pkl','rb'))

In [10]:
rpattern_values

{'S0022309300000570': [[], [], []],
 'S0022309300000661': [[], [], ['2.65', '2.75']],
 'S0022309300001034': [['0.2', '0.54', '0.20', '0.15', '2', '0.46'],
  ['0.3', '0.4', '0.6'],
  []],
 'S0022309300001241': [[], ['10'], []],
 'S0022309300001319': [[], [], []],
 'S0022309300001393': [[], [], []],
 'S0022309300001435': [['0.35', '1.0', '1', '20', '0.30', '0.20', '0.25'],
  [],
  []],
 'S0022309300001459': [[], [], []],
 'S0022309300001538': [['4', '10'], [], []],
 'S0022309300001629': [['20', '10', '5', '15', '25'], [], []],
 'S0022309300002301': [[], [], []],
 'S0022309300002313': [[], [], []],
 'S0022309300002453': [[], [], []],
 'S0022309300002714': [['5', '20', '40', '10'], [], []],
 'S0022309300002805': [['30', '20', '70', '69', '50', '60'],
  ['5', '20', '10'],
  []],
 'S0022309300002854': [['6', '14', '10'], [], []],
 'S0022309300003136': [['2'], [], []],
 'S0022309300003343': [[], [], []],
 'S0022309300003367': [[], [], []],
 'S0022309300003380': [[], [], []],
 'S00223093000034

In [11]:
MODEL_SAVE_PATH = '../scratch/mtp_trainClassifierWithout100_ratio1to6_run1_FlanT5Large.pt'
OUTPUT_FILE_PATH = 'mtp_predictions_test_Dataset_Classifier_ratio1to6.txt' 
GOLD_FILE_PATH = 'mtp_gold_test_Dataset_Classifier_ratio1to6.txt'

MODEL_NAME = 'google/flan-t5-large'
MODEL_LOAD_PATH = 'google/flan-t5-large'

In [12]:

max_input_length = 700
max_target_length = 10
batch_size = 4

f1_metric = load("f1")

def writeListToFile(lst, fname=OUTPUT_FILE_PATH):
    with open(fname, 'w', encoding='utf-8') as fp:
        for pred in lst:
            fp.write("%s\n" % pred)
    print(f"Written to file: {fname}")

X_test_df = pickle.load(open('test_dataset_complete_for_end2end_1415.pkl','rb'))
# print(X_test_df.columns)

X_test_df_compNonComp = X_test_df.rename(columns={"sentence":"Input", "is_comp":"Output"})


X_test = X_test_df_compNonComp[['Input', 'Output']]
# print(X_test.head())
lst = X_test['Output'].tolist()
writeListToFile(lst, GOLD_FILE_PATH)


device = 'cuda' if cuda.is_available() else 'cpu'
print(device)

tokenizer = T5Tokenizer.from_pretrained(MODEL_SAVE_PATH)

def preprocessTrainDev(examples):
    inputs = [doc for doc in examples["Input"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length ,padding='max_length')
    
    # contexts = tokenizer(examples["Context"], max_length=max_input_length, truncation=True,add_special_tokens=True,padding='max_length')
    # model_inputs["contexts"] = contexts
    # print("LABELS")
    
    outputs = [str(doc) for doc in examples["Output"]]
    labels = tokenizer(outputs, max_length=max_target_length, padding='max_length')
    # print(labels.shape)
    # print(labels)
    labelsInpIds = labels.input_ids
    # print(labelsInpIds)
    labelsInpIds = np.array(labelsInpIds)
    labelsInpIds[labelsInpIds == tokenizer.pad_token_id] = -100
    labelsInpIds = list(labelsInpIds)
    model_inputs["labels"] = labelsInpIds

    return model_inputs

# def preprocessTest(examples):
#     inputs = [doc for doc in examples["Input"]]
#     model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True,add_special_tokens=True,padding='max_length')

#     return model_inputs


datasetVal = Dataset.from_pandas(X_test)
tokenized_datasets_val = datasetVal.map(preprocessTrainDev, batched=True)


def generateTestOutput(model, testdata, tokenizer):
    model.eval()
    data = testdata['input_ids']
    amask = testdata['attention_mask']
    ds = Dataset.from_dict({"data":data, "attention_mask":amask}).with_format("torch")
    dataloader = DataLoader(ds, batch_size=1, shuffle=False)
    outputLst =[]
    # total_loss = 0.0
    
    with torch.no_grad():
        for i, (inputs) in enumerate(tqdm(dataloader)):
            input_ids = inputs['data'].to(device)
            amasks = inputs['attention_mask'].to(device)
            # generate model outputs
            generated_ids = model.generate(
                input_ids = input_ids,
                attention_mask = amasks,
                max_new_tokens=10,
                num_beams = 2)
            # print(generated_ids)
            output = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)

            outputLst.extend(output)
    writeListToFile(outputLst, OUTPUT_FILE_PATH)


model = T5ForConditionalGeneration.from_pretrained(MODEL_SAVE_PATH).to(device)

print("Validating from generate -->")
print(datetime.now())

generateTestOutput(model, tokenized_datasets_val, tokenizer)


print("GENERATION COMPLETED !!")


Written to file: mtp_gold_test_Dataset_Classifier_ratio1to6.txt
cuda


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Map:   0%|          | 0/1415 [00:00<?, ? examples/s]

Validating from generate -->
2024-04-16 17:48:49.608281


100%|███████████████████████████████████████████████████████████████████████████████| 1415/1415 [06:30<00:00,  3.62it/s]

Written to file: mtp_predictions_test_Dataset_Classifier_ratio1to6.txt
GENERATION COMPLETED !!


In [13]:
# X_val = pickle.load(open('test_sentences_for_comp-noncomp_classification_1496.pkl','rb'))
# OUTPUT_FILE_PATH = 'mtp_predictions_test_Dataset_Classifier_ratio1to6.txt' 
# GOLD_FILE_PATH = 'mtp_gold_test_Dataset_Classifier_ratio1to6.txt'

# print(X_val.columns)

analysis_eval = []
with open(GOLD_FILE_PATH, 'r', encoding='utf-8') as fread:
        gold = fread.readlines()
with open(OUTPUT_FILE_PATH, 'r', encoding='utf-8') as fp:
        pred = fp.readlines()

gold = [int(i[0]) for i in gold]
pred = [int(i[0]) for i in pred]

pred_df = pd.DataFrame(pred, columns=['Pred_CompNC'])
gold_df = pd.DataFrame(gold, columns=['Gold_CompNC'])


pred_df.reset_index(drop=True, inplace=True)
gold_df.reset_index(drop=True, inplace=True)
X_test_df.reset_index(drop=True, inplace=True)


new_df = pd.concat([X_test_df,pred_df, gold_df], axis=1)
# print(new_df.shape)
new_df.to_csv('analysis_classification_ratio1to6_Without100_testData_2.csv', encoding='utf-16', sep='|')
# print(new_df.shape)

pre_df = new_df[(new_df['Pred_CompNC']==1) & (new_df['Gold_CompNC']==1)]
rec_df = new_df[(new_df['Pred_CompNC']==0) & (new_df['Gold_CompNC']==1)]

tp = len(pre_df)
fn = len(rec_df)
tn = len(new_df[(new_df['Pred_CompNC']==0) & (new_df['Gold_CompNC']==0)])
fp = len(new_df[(new_df['Pred_CompNC']==1) & (new_df['Gold_CompNC']==0)])

print("TP = ",tp)
print("FP = ",fp)
print("TN = ",tn)
print("FN = ",fn)

precision = tp/(tp+fp)
recall = tp/(tp+fn)
f1 = 2 * precision * recall / (precision + recall)

print("Precision = ", precision)
print("Recall = ", recall)
print("F1 = ",f1)

TP =  93
FP =  23
TN =  1277
FN =  22
Precision =  0.8017241379310345
Recall =  0.808695652173913
F1 =  0.8051948051948051


In [14]:
new_df

,pii,sentence,comp_type,is_comp,is_SCC,gold_comps_for_pii,Pred_CompNC,Gold_CompNC
0,S0167273804001882,Electrical and electrochemical properties of a...,2,1,0,"[[(WO3, 45.0), (Ag2O, 45.0), (AgI, 7.5), (AgCl...",1,1
1,S0167273804001882,Investigations on electrical and electrochemic...,2,1,0,"[[(WO3, 45.0), (Ag2O, 45.0), (AgI, 7.5), (AgCl...",1,1
2,S0167273804001882,A “quenched [0.75AgI: 0.25AgCl] mixed system/s...,1,1,1,"[[(WO3, 45.0), (Ag2O, 45.0), (AgI, 7.5), (AgCl...",1,1
3,S0167273804001882,Compositional variation of room temperature co...,0,0,-1,"[[(WO3, 45.0), (Ag2O, 45.0), (AgI, 7.5), (AgCl...",0,0
4,S0167273804001882,The composition: 0.7[0.75AgI: 0.25AgCl]: 0.3[A...,1,1,1,"[[(WO3, 45.0), (Ag2O, 45.0), (AgI, 7.5), (AgCl...",1,1
...,...,...,...,...,...,...,...,...
1410,S0022309307011763,Through further considering the local co-ordin...,1,1,1,"[[(KCl, 66.67), (Ga2S3, 33.33)], [(CsCl, 66.67...",0,1
1411,S0022309307011763,The chemistry of Ga in alkali chloride melts i...,0,0,-1,"[[(KCl, 66.67), (Ga2S3, 33.33)], [(CsCl, 66.67...",0,0
1412,S0022309307011763,Raman scattering is a very useful vibrating sp...,0,0,-1,"[[(KCl, 66.67), (Ga2S3, 33.33)], [(CsCl, 66.67...",0,0
1413,S0022309307011763,"Therefore, understanding and utilization of th...",0,0,-1,"[[(KCl, 66.67), (Ga2S3, 33.33)], [(CsCl, 66.67...",0,0


In [15]:
X_test_for_comp = new_df[new_df['Pred_CompNC']==1]

In [16]:
X_test_for_comp

,pii,sentence,comp_type,is_comp,is_SCC,gold_comps_for_pii,Pred_CompNC,Gold_CompNC
0,S0167273804001882,Electrical and electrochemical properties of a...,2,1,0,"[[(WO3, 45.0), (Ag2O, 45.0), (AgI, 7.5), (AgCl...",1,1
1,S0167273804001882,Investigations on electrical and electrochemic...,2,1,0,"[[(WO3, 45.0), (Ag2O, 45.0), (AgI, 7.5), (AgCl...",1,1
2,S0167273804001882,A “quenched [0.75AgI: 0.25AgCl] mixed system/s...,1,1,1,"[[(WO3, 45.0), (Ag2O, 45.0), (AgI, 7.5), (AgCl...",1,1
4,S0167273804001882,The composition: 0.7[0.75AgI: 0.25AgCl]: 0.3[A...,1,1,1,"[[(WO3, 45.0), (Ag2O, 45.0), (AgI, 7.5), (AgCl...",1,1
14,S0167273804001882,"However, in a recent investigation Agrawal et ...",1,1,1,"[[(WO3, 45.0), (Ag2O, 45.0), (AgI, 7.5), (AgCl...",1,1
...,...,...,...,...,...,...,...,...
1339,S0022309307012021,The glass samples with the composition of Ge x...,2,1,0,"[[(Se, 80.0), (Ge, 15.0), (Ga, 5.0)], [(Se, 77...",1,1
1340,S0022309307012021,"In the present paper, a series of dysprosium d...",2,1,0,"[[(Se, 80.0), (Ge, 15.0), (Ga, 5.0)], [(Se, 77...",1,1
1351,S0022309307012021,Dysprosium doped Ge x Ga5Se(95−x) (x =15–30) c...,2,1,0,"[[(Se, 80.0), (Ge, 15.0), (Ga, 5.0)], [(Se, 77...",1,1
1393,S0022309307011763,Based on the analysis of the local co-ordinati...,1,1,1,"[[(KCl, 66.67), (Ga2S3, 33.33)], [(CsCl, 66.67...",1,1


In [17]:

# MODEL_SAVE_PATH = '../scratch/mtp_trainClassifierWithout100_ratio1to6_run1_FlanT5Large.pt'
# OUTPUT_FILE_PATH = 'mtp_predictions_test_Dataset_Classifier_ratio1to6.txt' 
# GOLD_FILE_PATH = 'mtp_gold_test_Dataset_Classifier_ratio1to6.txt'
RUN_NUM = 1
RatioRunName = f'dm_vs_eqn_run{RUN_NUM}'
TASK = 'dm_vs_eqn'
MODEL_SAVE_PATH = f'../scratch/mtp_trainClassifierWithout100_{RatioRunName}_FlanT5Large.pt'
OUTPUT_FILE_PATH = f'mtp_predictions_trainClassifierWithout100_{RatioRunName}_FlanT5Large.txt' 
GOLD_FILE_PATH = f'mtp_gold_trainClassifierWithout100_{RatioRunName}_FlanT5Large.txt'


MODEL_NAME = 'google/flan-t5-large'
MODEL_LOAD_PATH = 'google/flan-t5-large'
max_input_length = 700
max_target_length = 10
batch_size = 4


X_test_for_comp_SCCMCC = X_test_for_comp.rename(columns={"sentence":"Input", "is_SCC":"Output"})

X_test = X_test_for_comp_SCCMCC[['Input', 'Output']]
lst = X_test['Output'].tolist()
writeListToFile(lst, GOLD_FILE_PATH)


device = 'cuda' if cuda.is_available() else 'cpu'
print(device)

tokenizer = T5Tokenizer.from_pretrained(MODEL_SAVE_PATH)
datasetVal = Dataset.from_pandas(X_test)
tokenized_datasets_val = datasetVal.map(preprocessTrainDev, batched=True)

model = T5ForConditionalGeneration.from_pretrained(MODEL_SAVE_PATH).to(device)

print("Validating from generate -->")
print(datetime.now())

generateTestOutput(model, tokenized_datasets_val, tokenizer)


print("GENERATION COMPLETED !!")

Written to file: mtp_gold_trainClassifierWithout100_dm_vs_eqn_run1_FlanT5Large.txt
cuda


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Map:   0%|          | 0/116 [00:00<?, ? examples/s]

Validating from generate -->
2024-04-16 17:55:42.951718


100%|█████████████████████████████████████████████████████████████████████████████████| 116/116 [00:25<00:00,  4.49it/s]

Written to file: mtp_predictions_trainClassifierWithout100_dm_vs_eqn_run1_FlanT5Large.txt
GENERATION COMPLETED !!


## ANALYSIS OF PREDICTED AS SCC/MCC

In [18]:
OUTPUT_FILE_PATH

'mtp_predictions_trainClassifierWithout100_dm_vs_eqn_run1_FlanT5Large.txt'

In [19]:
s = '-1/n'
int(s[:-2])

-1

In [20]:

# OUTPUT_FILE_PATH = 'mtp_predictions_trainClassifierWithout100_1.txt' 
# GOLD_FILE_PATH = 'mtp_gold_trainClassifierWithout100_ratio1to4.txt'
# X_val = pickle.load(open('val_data_direct_match_vs_eqn_classification_1455.pkl','rb'))
# GOLD_FILE_PATH  = 'mtp_gold_trainClassifierWithout100_dm_vs_eqn_run2_FlanT5Large.txt'
# OUTPUT_FILE_PATH = 'mtp_predictions_trainClassifierWithout100_dm_vs_eqn_run2_FlanT5Large.txt'

# print(X_val.columns)

analysis_eval = []
with open(GOLD_FILE_PATH, 'r', encoding='utf-8') as fread:
        gold_str = fread.readlines()
with open(OUTPUT_FILE_PATH, 'r', encoding='utf-8') as fp:
        pred_str = fp.readlines()


# print(gold)
# print(pred)

# for i in gold:
#     print(i)

gold = [int(i) for i in gold_str]
pred = [int(i) for i in pred_str]

# print(gold)
# print(pred)


new_df = pd.concat([X_test_for_comp.reset_index(),pd.DataFrame(pred, columns=['Pred_SCC_MCC']).reset_index(),
                    pd.DataFrame(gold, columns=['Gold_SCC_MCC']).reset_index()], axis=1)

# print(new_df)
# new_df['Pred'] = new_df['Pred'].apply(lambda x: x.replace('\n', ''))
# new_df['Gold'] = new_df['Gold'].apply(lambda x: x.replace('\n', ''))
# new_df['score'] = new_df['score'].apply(lambda x: x.replace('\n', ''))
new_df.to_csv(f'analaysis_{OUTPUT_FILE_PATH}.csv', encoding='utf-16', sep='|')
# new_df.to_csv(f'analysis_{OUTPUT_FILE_PATH}.csv', encoding='utf-16')


pre_df = new_df[(new_df['Pred_SCC_MCC']==1) & (new_df['Gold_SCC_MCC']==1)]
rec_df = new_df[(new_df['Pred_SCC_MCC']==0) & (new_df['Gold_SCC_MCC']==1)]

tp = len(pre_df)
fn = len(rec_df)
tn = len(new_df[(new_df['Pred_SCC_MCC']==0) & (new_df['Gold_SCC_MCC']==0)])
fp = len(new_df[(new_df['Pred_SCC_MCC']==1) & (new_df['Gold_SCC_MCC']==0)])

print("TP = ",tp)
print("FP = ",fp)
print("TN = ",tn)
print("FN = ",fn)

precision = tp/(tp+fp)
recall = tp/(tp+fn)
f1 = 2 * precision * recall / (precision + recall)

print("Precision = ", precision)
print("Recall = ", recall)
print("F1 = ",f1)

TP =  70
FP =  2
TN =  21
FN =  0
Precision =  0.9722222222222222
Recall =  1.0
F1 =  0.9859154929577464


In [21]:

# OUTPUT_FILE_PATH = 'mtp_predictions_trainClassifierWithout100_1.txt' 
# GOLD_FILE_PATH = 'mtp_gold_trainClassifierWithout100_ratio1to4.txt'
# X_val = pickle.load(open('val_data_direct_match_vs_eqn_classification_1455.pkl','rb'))
# GOLD_FILE_PATH  = 'mtp_gold_trainClassifierWithout100_dm_vs_eqn_run2_FlanT5Large.txt'
# OUTPUT_FILE_PATH = 'mtp_predictions_trainClassifierWithout100_dm_vs_eqn_run2_FlanT5Large.txt'

# print(X_val.columns)

analysis_eval = []
with open(GOLD_FILE_PATH, 'r', encoding='utf-8') as fread:
        gold_str = fread.readlines()
with open(OUTPUT_FILE_PATH, 'r', encoding='utf-8') as fp:
        pred_str = fp.readlines()


# print(gold)
# print(pred)

# for i in gold:
#     print(i)

gold = [int(i) for i in gold_str]
pred = [int(i) for i in pred_str]

# print(gold)
# print(pred)


new_df = pd.concat([X_test_for_comp.reset_index(drop=True),pd.DataFrame(pred, columns=['Pred_SCC_MCC']).reset_index(drop=True),
                    pd.DataFrame(gold, columns=['Gold_SCC_MCC']).reset_index(drop=True)], axis=1)

# print(new_df)
# new_df['Pred'] = new_df['Pred'].apply(lambda x: x.replace('\n', ''))
# new_df['Gold'] = new_df['Gold'].apply(lambda x: x.replace('\n', ''))
# new_df['score'] = new_df['score'].apply(lambda x: x.replace('\n', ''))
new_df.to_csv(f'analaysis_{OUTPUT_FILE_PATH}.csv', encoding='utf-16', sep='|')
# new_df.to_csv(f'analysis_{OUTPUT_FILE_PATH}.csv', encoding='utf-16')


pre_df = new_df[(new_df['Pred_SCC_MCC']==1) & (new_df['Gold_SCC_MCC']==1)]
rec_df = new_df[(new_df['Pred_SCC_MCC']==0) & (new_df['Gold_SCC_MCC']==1)]

tp = len(pre_df)
fn = len(rec_df)
tn = len(new_df[(new_df['Pred_SCC_MCC']==0) & (new_df['Gold_SCC_MCC']==0)])
fp = len(new_df[(new_df['Pred_SCC_MCC']==1) & (new_df['Gold_SCC_MCC']!=1)])

print("TP = ",tp)
print("FP = ",fp)
print("TN = ",tn)
print("FN = ",fn)

precision = tp/(tp+fp)
recall = tp/(tp+fn)
f1 = 2 * precision * recall / (precision + recall)

print("Precision = ", precision)
print("Recall = ", recall)
print("F1 = ",f1)

TP =  70
FP =  25
TN =  21
FN =  0
Precision =  0.7368421052631579
Recall =  1.0
F1 =  0.8484848484848484


## ANALYSIS OF ALL AS SCC/MCC

In [22]:
new_df

,pii,sentence,comp_type,is_comp,is_SCC,gold_comps_for_pii,Pred_CompNC,Gold_CompNC,Pred_SCC_MCC,Gold_SCC_MCC
0,S0167273804001882,Electrical and electrochemical properties of a...,2,1,0,"[[(WO3, 45.0), (Ag2O, 45.0), (AgI, 7.5), (AgCl...",1,1,0,0
1,S0167273804001882,Investigations on electrical and electrochemic...,2,1,0,"[[(WO3, 45.0), (Ag2O, 45.0), (AgI, 7.5), (AgCl...",1,1,0,0
2,S0167273804001882,A “quenched [0.75AgI: 0.25AgCl] mixed system/s...,1,1,1,"[[(WO3, 45.0), (Ag2O, 45.0), (AgI, 7.5), (AgCl...",1,1,1,1
3,S0167273804001882,The composition: 0.7[0.75AgI: 0.25AgCl]: 0.3[A...,1,1,1,"[[(WO3, 45.0), (Ag2O, 45.0), (AgI, 7.5), (AgCl...",1,1,1,1
4,S0167273804001882,"However, in a recent investigation Agrawal et ...",1,1,1,"[[(WO3, 45.0), (Ag2O, 45.0), (AgI, 7.5), (AgCl...",1,1,1,1
...,...,...,...,...,...,...,...,...,...,...
111,S0022309307012021,The glass samples with the composition of Ge x...,2,1,0,"[[(Se, 80.0), (Ge, 15.0), (Ga, 5.0)], [(Se, 77...",1,1,0,0
112,S0022309307012021,"In the present paper, a series of dysprosium d...",2,1,0,"[[(Se, 80.0), (Ge, 15.0), (Ga, 5.0)], [(Se, 77...",1,1,0,0
113,S0022309307012021,Dysprosium doped Ge x Ga5Se(95−x) (x =15–30) c...,2,1,0,"[[(Se, 80.0), (Ge, 15.0), (Ga, 5.0)], [(Se, 77...",1,1,0,0
114,S0022309307011763,Based on the analysis of the local co-ordinati...,1,1,1,"[[(KCl, 66.67), (Ga2S3, 33.33)], [(CsCl, 66.67...",1,1,1,1


## EXTRACT SCC

In [23]:
X_val_extract_SCC = new_df[new_df['Pred_SCC_MCC'] == 1]
X_val_extract_MCC = new_df[new_df['Pred_SCC_MCC'] == 0]

In [24]:
X_val_extract_SCC

,pii,sentence,comp_type,is_comp,is_SCC,gold_comps_for_pii,Pred_CompNC,Gold_CompNC,Pred_SCC_MCC,Gold_SCC_MCC
2,S0167273804001882,A “quenched [0.75AgI: 0.25AgCl] mixed system/s...,1,1,1,"[[(WO3, 45.0), (Ag2O, 45.0), (AgI, 7.5), (AgCl...",1,1,1,1
3,S0167273804001882,The composition: 0.7[0.75AgI: 0.25AgCl]: 0.3[A...,1,1,1,"[[(WO3, 45.0), (Ag2O, 45.0), (AgI, 7.5), (AgCl...",1,1,1,1
4,S0167273804001882,"However, in a recent investigation Agrawal et ...",1,1,1,"[[(WO3, 45.0), (Ag2O, 45.0), (AgI, 7.5), (AgCl...",1,1,1,1
6,S0167273804001882,Hosono et al. [11] have recently carried out a...,1,1,1,"[[(WO3, 45.0), (Ag2O, 45.0), (AgI, 7.5), (AgCl...",1,1,1,1
8,S0167273804001882,The composition: 0.7[0.75AgI: 0.25AgCl]: 0.3[A...,1,1,1,"[[(WO3, 45.0), (Ag2O, 45.0), (AgI, 7.5), (AgCl...",1,1,1,1
...,...,...,...,...,...,...,...,...,...,...
103,S0022309305000608,"In this way, it is found that Ge4, GeI2S and G...",1,1,1,"[[(S, 25.0), (I, 75.0)], [(Ge, 50.0), (I, 50.0...",1,1,1,1
106,S0022309305000608,Ge 4 : This is a pyramidal system with Ge–Ge d...,0,0,-1,"[[(S, 25.0), (I, 75.0)], [(Ge, 50.0), (I, 50.0...",1,0,1,-1
107,S002230930300379X,2.1 Glass preparation The glass compositions u...,2,1,0,"[[(BaF2, 30.0), (ZrF4, 62.0), (LaF3, 8.0)], [(...",1,1,1,0
114,S0022309307011763,Based on the analysis of the local co-ordinati...,1,1,1,"[[(KCl, 66.67), (Ga2S3, 33.33)], [(CsCl, 66.67...",1,1,1,1


In [25]:
X_val_extract_MCC

,pii,sentence,comp_type,is_comp,is_SCC,gold_comps_for_pii,Pred_CompNC,Gold_CompNC,Pred_SCC_MCC,Gold_SCC_MCC
0,S0167273804001882,Electrical and electrochemical properties of a...,2,1,0,"[[(WO3, 45.0), (Ag2O, 45.0), (AgI, 7.5), (AgCl...",1,1,0,0
1,S0167273804001882,Investigations on electrical and electrochemic...,2,1,0,"[[(WO3, 45.0), (Ag2O, 45.0), (AgI, 7.5), (AgCl...",1,1,0,0
5,S0167273804001882,The present paper reports another Ag+ ion cond...,2,1,0,"[[(WO3, 45.0), (Ag2O, 45.0), (AgI, 7.5), (AgCl...",1,1,0,0
7,S0167273804001882,"3.1 Compositional variation of σ, μ, n and pha...",2,1,0,"[[(WO3, 45.0), (Ag2O, 45.0), (AgI, 7.5), (AgCl...",1,1,0,0
10,S0167273804001882,Fig. 2 shows ‘log μ−x’ and ‘log n−x’ plots for...,2,1,0,"[[(WO3, 45.0), (Ag2O, 45.0), (AgI, 7.5), (AgCl...",1,1,0,0
23,S002230939900321X,The sample compositions are stoichiometric and...,2,1,0,"[[(As, 10.0), (S, 65.0), (Ge, 25.0)], [(As, 8....",1,1,0,0
30,S0022309305000761,"Here, an effort has been made to investigate v...",2,1,0,"[[(Te, 20.0), (I, 80.0)], [(Te, 30.0), (I, 70....",1,1,0,0
51,S0022309309003883,Experiments performed on Ge1- x S x glasses (w...,2,1,0,"[[(S, 72.0), (Ge, 28.0)], [(S, 71.0), (Ge, 27....",1,1,0,0
81,S0022309307008460,"In the As40S60− x Se x glasses, the main struc...",2,1,0,"[[(As, 40.0), (S, 60.0)], [(As, 24.0), (Se, 38...",1,1,0,0
82,S0022309307008460,"For chalcogen-rich As–S–Se glasses, the molecu...",2,1,0,"[[(As, 40.0), (S, 60.0)], [(As, 24.0), (Se, 38...",1,1,0,0


In [26]:
print("Predicted as SCC = ",len(X_val_extract_SCC))
print("Predicted as MCC = ",len(X_val_extract_MCC))


Predicted as SCC =  95
Predicted as MCC =  21


In [29]:
import ast

In [30]:
TASK = 'end2end_CompExtractor_Without100_FlanT5Large_DirectMatch_NB'
RUNNUM = 'run_1'

MODEL_SAVE_PATH = f'../scratch/mtp_CompExtractor_Without100_FlanT5Large_DirectMatch_run_2.pt'
OUTPUT_FILE_PATH = f'mtp_predictions_{TASK}_{RUNNUM}.txt' 
GOLD_FILE_PATH = f'mtp_gold_{TASK}.txt'

MODEL_NAME = 'google/flan-t5-large'
MODEL_LOAD_PATH = 'google/flan-t5-large'


max_input_length = 300
max_target_length = 800


exact_match_metric = load("exact_match")


X_val_extract = X_val_extract_SCC.rename(columns={'sentence': 'Input', 'gold_comps_for_pii':'Output'})


X_val_extract['Output'] = X_val_extract['Output'].apply(str)
X_val_extract = X_val_extract[['Input', 'Output']]


lst = X_val_extract['Output'].tolist()
writeListToFile(lst, GOLD_FILE_PATH)


device = 'cuda' if cuda.is_available() else 'cpu'
print(device)

tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)
model = T5ForConditionalGeneration.from_pretrained(MODEL_SAVE_PATH).to(device)


def preprocessTrainDev(examples):
    inputs = [doc for doc in examples["Input"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True,add_special_tokens=True ,padding='max_length')
    labels = tokenizer(examples["Output"], max_length=max_target_length, truncation=True,add_special_tokens=True, padding='max_length')
    labelsInpIds = labels.input_ids
    
    labelsInpIds = np.array(labelsInpIds)
    labelsInpIds[labelsInpIds == tokenizer.pad_token_id] = -100
    labelsInpIds = list(labelsInpIds)
    model_inputs["labels"] = labelsInpIds

    return model_inputs

datasetVal = Dataset.from_pandas(X_val_extract)
tokenized_datasets_val = datasetVal.map(preprocessTrainDev, batched=True)


def generateTestOutput(model, testdata, tokenizer):
    model.eval()
    data = testdata['input_ids']
    amask = testdata['attention_mask']
    ds = Dataset.from_dict({"data":data, "attention_mask":amask}).with_format("torch")
    dataloader = DataLoader(ds, batch_size=8, shuffle=False)
    outputLst =[]
    # total_loss = 0.0
    
    with torch.no_grad():
        for i, (inputs) in enumerate(tqdm(dataloader)):
            input_ids = inputs['data'].to(device)
            amasks = inputs['attention_mask'].to(device)
            # generate model outputs
            generated_ids = model.generate(
                input_ids = input_ids,
                attention_mask = amasks,
                max_new_tokens=800,
                num_beams = 1)
            # print(generated_ids)
            output = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)

            outputLst.extend(output)
    writeListToFile(outputLst, OUTPUT_FILE_PATH)



print("Generating extraction predictions -->")
print(datetime.now())
generateTestOutput(model, tokenized_datasets_val, tokenizer)


print("Starting evaluations ----------->")

def evaluationScore(goldPath, predPath):
    with open(goldPath, 'r', encoding='utf-8') as fread:
        gold = fread.readlines()
    with open(predPath, 'r', encoding='utf-8') as fp:
        pred = fp.readlines()
    result = exact_match_metric.compute(predictions=pred, references=gold)
    return result

# print("Validation ended.. Calculating score -->")
# print(datetime.now())

print("Exact match score --->")
print(evaluationScore(GOLD_FILE_PATH, OUTPUT_FILE_PATH))

# EVALUATION 2 ----------->
def evaluationScore(goldPath, predPath):
    with open(goldPath, 'r', encoding='utf-8') as fread:
        gold = fread.readlines()
    with open(predPath, 'r', encoding='utf-8') as fp:
        pred = fp.readlines()
    
    result = 0
    ct=0
    for g,p in zip(gold, pred):
        # print("GOLD")
        # print(g)
        # print("PRED")
        # print(p)
        evalscore = evaluateComposition(g, p)
        # print(evalscore)
        result += evalscore
        # print(result)
        ct+=1
    # result = exact_match_metric.compute(predictions=pred, references=gold)
    return result/ct

def evaluateComposition(gold, predicted):
    
    gold = ast.literal_eval(gold)
    try:
        predicted = ast.literal_eval(predicted)
    except:
        # print("Cannot parse")
        # print(predicted)
        return 0.0
    gold_sets = [set(compound_list) for compound_list in gold]
    predicted_sets = [set(compound_list) for compound_list in predicted]
    
    # print("INSIDE EC")
    # print(gold_sets)
    # print(predicted_sets)
    # print("CLOSE")
    
    precision=0
    recall=0

    for predicted_set in predicted_sets:
        if(predicted_set in gold_sets):
            precision+=1
    
    for gold_set in gold_sets:
        if(gold_set in predicted_sets):
            recall+=1   
    
    precision /= len(predicted_sets)
    recall /= len(gold_sets)
    
    if(precision==0 and recall ==0):
        f1=0.0
    else: 
        f1 = (2*precision*recall)/(precision+recall)
    
    return round(f1,2)


print("Metric score --->")
print(evaluationScore(GOLD_FILE_PATH, OUTPUT_FILE_PATH))

Written to file: mtp_gold_end2end_CompExtractor_Without100_FlanT5Large_DirectMatch_NB.txt
cuda


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Map:   0%|          | 0/95 [00:00<?, ? examples/s]

Generating extraction predictions -->
2024-04-16 17:58:32.770695


100%|███████████████████████████████████████████████████████████████████████████████████| 12/12 [00:34<00:00,  2.87s/it]

Written to file: mtp_predictions_end2end_CompExtractor_Without100_FlanT5Large_DirectMatch_NB_run_1.txt
Starting evaluations ----------->
Exact match score --->
{'exact_match': 0.11578947368421053}
Metric score --->
0.2722105263157894


In [31]:
# with open(OUTPUT_FILE_PATH, 'r', encoding='utf-8') as fp:
#     pred = fp.readlines()


# pred_extract_df = pd.DataFrame(pred, columns=['PredExtract'])
# extract_df['PredExtract'] = extract_df['PredExtract'].apply(lambda x: x.replace('\n',''))
# extract_df['PredExtract'] = extract_df['PredExtract'].apply(str)
# extract_df['PredExtract'] = extract_df['PredExtract'].apply(lambda x: ast.literal_eval(x))


# # final_df = pd.concat([X_val, pred_extract_df], axis=1)
# X_val_extracted = extract_df[extract_df['pred'] == '1']
# pred_combined = pd.concat([X_val, pred_extract_df], axis=1)
# X_val_remaining = extract_df[extract_df['pred'] == '0']

# final_df = pd.concat([X_val, pred_extract_df], axis=0)


# def final_evaluation(goldLst, predLst):
#     result = 0
#     ct=0
#     for g,p in zip(goldLst, predLst):
#         # print("GOLD")
#         # print(g)
#         # print("PRED")
#         # print(p)
#         evalscore = evaluateComposition(g, p)
#         # print(evalscore)
#         result += evalscore
#         # print(result)
#         ct+=1
#     # result = exact_match_metric.compute(predictions=pred, references=gold)
#     return result/ct

# print("FINAL RESULT --->>>")
# print(final_evaluation(final_df['composition'].tolist(), final_df['PredExtract'].tolist()))



## EXTRACT MCC

In [34]:
import ast
import operator as op
operators = {ast.Add: op.add, ast.Sub: op.sub, ast.Mult: op.mul, ast.Div: op.truediv}

def eval_(node):
    if isinstance(node, ast.Num):
        return node.n
    elif isinstance(node, ast.BinOp):
        return operators[type(node.op)](eval_(node.left), eval_(node.right))
    else:
        raise TypeError(node)

def eval_expr(expr):
    return eval_(ast.parse(expr, mode='eval').body)


def norm_sum_to_1(composition, sum_perc):
    assert len(composition) > 0
    corr_comp = []
    for c in composition:
        try:
            val = eval_expr(c[1])
            if val < 0: return []
            if val > 0: corr_comp.append(c)
        except:
            corr_comp.append(c)
    norm_comp = [(c[0], f'({c[1]})/({sum_perc})') for c in corr_comp]
    d = dict()
    for c in norm_comp:
        if c[0] not in d:
            d[c[0]] = c[1].replace(' ', '')
        else:
            d[c[0]] += '+' + c[1].replace(' ', '')
    return [(k, v) for k, v in d.items()]

In [135]:
from itertools import product
import re
def substituteRegex(compounds, regexVals, span):
    # Given list of compounds and expressions
    # compounds = [('GeS2', '(1-x)/(1-x+x+y)'), ('Ga2S3', '(x)/(1-x+x+y)'), ('Al2O3', 0.5)]

    # print(compounds)
    is_xyz_present = any([bool(re.search('[xyz]', i[1])) for i in compounds if(isinstance(i[1], str))])
    # print('Is XYZ ? =', is_xyz_present)
    if(is_xyz_present == False):
        # print(compounds)
        new_comp = []
        for i in compounds:
            # print(i)
            new_comp.append((i[0], float(i[1])))
        # try:
            # new_comp = ast.literal_eval(compounds)
        # except:
            # new_comp = []
            # print("AST LITERAL FAILED FOR = ",compounds)
        return [(new_comp,span)]

    # Given lists of possible values for x, y, and z
    # x_values = [0.1,0.2]
    # y_values = [0.4]
    # z_values = []
    # span = (42,48)
    x_values = regexVals[0]
    y_values = regexVals[1]
    z_values = regexVals[2]
    

    # Create a list to store the final result in the specified format
    result_list = []
    neg_subs_flag = 0
    # Generate all combinations of x, y, and z values, including the empty x and y
    combinations = list(product(x_values or [None], y_values or [None], z_values or [None]))
    # print(combinations)
    # Loop through each combination of x, y, and z values
    for x, y, z in combinations:
        # print(f'x={x}')
        # Create a list to store substituted expressions for each compound
        substituted_expressions = []
        
        
        # Loop through each compound and substitute values
        for compound in compounds:
            compound_name, expression = compound
            
            # Handle None value for x, y, and z when respective value lists are empty
            x_placeholder = str(x) if x is not None else 'x'
            y_placeholder = str(y) if y is not None else 'y'
            z_placeholder = str(z) if z is not None else 'z'
            
            substituted_expression = str(expression).replace('x', x_placeholder).replace('y', y_placeholder).replace('z', z_placeholder)
            try:
                substituted_expression = eval_expr(substituted_expression)
                # if(regexVals == [['72', '69', '66', '62', '75'], [], []]):
                #     print(f"SUBS EXPRESSION = {substituted_expression}")
                if(substituted_expression<0):
                    neg_subs_flag = 1
                    substituted_expression = 100 + substituted_expression - 1
            except TypeError as e:
                pass
            substituted_expressions.append((compound_name, substituted_expression))
            
            
            # else:
            #     substituted_expressions=expression
            # substituted_expressions.append((compound_name, substituted_expression))
        # if(len(substituted_expressions)==0):
        #     substituted_expressions = compounds
        # if(substituted_expressions!=compounds):
        result_list.append(substituted_expressions)

    # print(result_list)
    new_res_lst_neg = []
    if(neg_subs_flag == 1):
        for comp in result_list:
            new_comp_neg = []
            for cname, exp in comp:
                new_comp_neg.append((cname, exp/100))
            new_res_lst_neg.append(new_comp_neg)
        result_list = new_res_lst_neg
        neg_subs_flag = 0

        # print(new_res_lst_neg)

    # print(result_list)
    # total = sum([value for _, value in result_list])
    # normalized_values = [(name, value / total) for name, value in result_list]
    # print(normalized_values)
    
    # for combination in result_list:
    #     print(combination)  
    
    final_result = []    
    # Print the resulting list in the specified format
    for combination in result_list:
        # print(combination)
        try:
            total = sum([value for _, value in combination])
            # print(total)
            normalized_values = [(name, value / total) for name, value in combination]
            if(normalized_values <0):
                print(f'comb={combination}')
            final_result.append((normalized_values, span))
            # print(normalized_values)
        except TypeError as e:
            # print(sys.exc_info()[0])
            final_result.append((combination, span))

    # print(final_result)
    return final_result


In [136]:
# def multiply100(lstOfLst):
#     print(lstOfLst)
#     if(len(lstOfLst)==0):
#         return [[]]
#     newLstOfLst = []
#     for compLst in lstOfLst:
#         newCompLst = []
#         for tuples in compLst:
#             if(tuples[1]!=0.0):
#                 newCompLst.append((tuples[0], tuples[1]*100.0))
#         newLstOfLst.append(newCompLst)
#     return newLstOfLst

In [137]:
def cleanParsedCompositions(parsedCompositions):
    # print("pc")
    # print(parsedCompositions)
    unique_data = [list(t) for t in set(tuple(i) for i in parsedCompositions)]
    # print("ud")
    # print(unique_data)
    new_unique_data = []
    for compLst in unique_data:
        new_compLst = []
        for tuples in compLst:
            # print(tuples)
            new_compLst.append((tuples[0], round(float(tuples[1])*100.0,2)))
        new_unique_data.append(new_compLst)
    return new_unique_data

In [138]:
def subsXYZ(pii, parsed):
    parsedCompositions = []
    rp = rpattern_values[pii]
    # print(rp)
    subsComp = []
    if(len(rp[0]) ==0 and len(rp[1]) ==0 and len(rp[0]) ==0):
        rp[0]=['0']
        rp[1]=['0']
        rp[2]=['0']
    # print("PII ----> ",pii)
    for i in parsed:
        # print("i= ",i)
        try:
            subs = substituteRegex(i, rp, ())
            # print("subs = ",subs)
            for comp in subs:
                parsedCompositions.append(comp[0])
        except:
            print("Failed for ",i)
    return cleanParsedCompositions(parsedCompositions)

In [139]:
rpattern_values['S0167273804001882']

[['0.7'], [], []]

In [140]:
subsXYZ("S0022309314000970", [[('As', '(40)/(40+60-x+x)'), ('S', '(60-x)/(40+60-x+x)'), ('Se', '(x)/(40+60-x+x)')], [('As', 0.4), ('S', 0.6)]])

[[('As', 40.0), ('S', 59.8), ('Se', 0.2)],
 [('As', 40.0), ('S', 60.0)],
 [('As', 40.0), ('S', 59.0), ('Se', 1.0)],
 [('As', 40.0), ('S', 59.3), ('Se', 0.7)],
 [('As', 40.0), ('S', 59.6), ('Se', 0.4)]]

In [141]:
subsXYZ("S0022309309003883", [[('Ge', '(1-x)/(1-x+x)'), ('S', '(x)/(1-x+x)')]])

[[('Ge', 34.0), ('S', 66.0)],
 [('Ge', 28.0), ('S', 72.0)],
 [('Ge', 31.0), ('S', 69.0)],
 [('Ge', 38.0), ('S', 62.0)],
 [('Ge', 25.0), ('S', 75.0)]]

In [152]:
TASK = 'CompExtractor_Without100_FlanT5Large_OnlyEqn'
RUNNUM = 'run_1'

# TRAIN_DATA_LOAD_PATH = 'train_data_for_equation_composition_extraction_1744.pkl'
# VAL_DATA_LOAD_PATH = 'val_data_for_equation_composition_extraction_416.pkl'


MODEL_SAVE_PATH = f'../scratch/mtp_{TASK}_{RUNNUM}.pt'
OUTPUT_FILE_PATH = f'mtp_predictions_{TASK}_{RUNNUM}_NB.txt' 
GOLD_FILE_PATH = f'mtp_gold_{TASK}_NB.txt'

# TASK = 'end2end_CompExtractor_Without100_FlanT5Large_DirectMatch'
# RUNNUM = 'run_1'

# MODEL_SAVE_PATH = f'../scratch/mtp_CompExtractor_Without100_FlanT5Large_DirectMatch_run_2.pt'
# OUTPUT_FILE_PATH = f'mtp_predictions_{TASK}_{RUNNUM}.txt' 
# GOLD_FILE_PATH = f'mtp_gold_{TASK}.txt'

MODEL_NAME = 'google/flan-t5-large'
# MODEL_LOAD_PATH = 'google/flan-t5-large'


max_input_length = 300
max_target_length = 800


exact_match_metric = load("exact_match")


X_val_extract = X_val_extract_MCC.rename(columns={'sentence': 'Input', 'gold_comps_for_pii':'Output'})
X_val_extract['Output'] = X_val_extract['Output'].apply(str)
X_val_extract = X_val_extract[['Input', 'Output']]

pii_lst = X_val_extract_MCC['pii'].tolist()

# X_val_extract_SCC['Output'] = X_val_extract_SCC['Output'].apply(str)

# def concatPrompt(df):
#     return str(df['Input']) + ' <SEP> ' + str(df['x']) + ' <SEP> ' + str(df['y']) + ' <SEP> ' + str(df['z'])
# X_val_extract['Input'] = X_val_extract.apply(lambda x: concatPrompt(x), axis=1)

# X_val_extract = X_val_extract_SCC[['Input', 'Output']]


lst = X_val_extract['Output'].tolist()
writeListToFile(lst, GOLD_FILE_PATH)


device = 'cuda' if cuda.is_available() else 'cpu'
print(device)

tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)
model = T5ForConditionalGeneration.from_pretrained(MODEL_SAVE_PATH).to(device)


def preprocessTrainDev(examples):
    inputs = [doc for doc in examples["Input"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True,add_special_tokens=True ,padding='max_length')
    labels = tokenizer(examples["Output"], max_length=max_target_length, truncation=True,add_special_tokens=True, padding='max_length')
    labelsInpIds = labels.input_ids
    
    labelsInpIds = np.array(labelsInpIds)
    labelsInpIds[labelsInpIds == tokenizer.pad_token_id] = -100
    labelsInpIds = list(labelsInpIds)
    model_inputs["labels"] = labelsInpIds

    return model_inputs

datasetVal = Dataset.from_pandas(X_val_extract)
tokenized_datasets_val = datasetVal.map(preprocessTrainDev, batched=True)


def generateTestOutput(model, testdata, tokenizer):
    model.eval()
    data = testdata['input_ids']
    amask = testdata['attention_mask']
    ds = Dataset.from_dict({"data":data, "attention_mask":amask}).with_format("torch")
    dataloader = DataLoader(ds, batch_size=8, shuffle=False)
    outputLst =[]
    # total_loss = 0.0
    
    with torch.no_grad():
        for i, (inputs) in enumerate(tqdm(dataloader)):
            input_ids = inputs['data'].to(device)
            amasks = inputs['attention_mask'].to(device)
            # generate model outputs
            generated_ids = model.generate(
                input_ids = input_ids,
                attention_mask = amasks,
                max_new_tokens=800,
                num_beams = 1)
            # print(generated_ids)
            output = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
            
            outputLst.extend(output)
    writeListToFile(outputLst, OUTPUT_FILE_PATH)



print("Generating extraction predictions -->")
print(datetime.now())
generateTestOutput(model, tokenized_datasets_val, tokenizer)


Written to file: mtp_gold_CompExtractor_Without100_FlanT5Large_OnlyEqn_NB.txt
cuda


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Map:   0%|          | 0/21 [00:00<?, ? examples/s]

Generating extraction predictions -->
2024-04-16 18:37:22.193355


100%|█████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:11<00:00,  3.83s/it]

Written to file: mtp_predictions_CompExtractor_Without100_FlanT5Large_OnlyEqn_run_1_NB.txt


In [153]:
rpattern_values['S002230939900321X']

[['0'], ['0'], ['0']]

In [154]:
with open(OUTPUT_FILE_PATH, 'r', encoding='utf-8') as fp:
    pred_lst = fp.readlines()
subs_pred = []

for pii, parsed in zip(pii_lst, pred_lst):
    try:
        parsed_ast =  ast.literal_eval(parsed)
    except:
        print("IDHAR")
        print(parsed)
        parsed_ast = [[]]
    subs_pred.append(subsXYZ(pii, parsed_ast))
    # print("PII = ",pii," PARSED = ",subs_pred)

IDHAR
[[('GeS2', '((1.0)*(5), '((1.0)*(1-x))/(5+2)'), ('As', '(((1-x)/(1-x+x))*(5))/(5+2)'), ('Ga', '(((x)/(1-x+x))*(5))/(5+2)')]]



In [155]:
len(subs_pred)

21

In [156]:
writeListToFile(subs_pred, OUTPUT_FILE_PATH)

Written to file: mtp_predictions_CompExtractor_Without100_FlanT5Large_OnlyEqn_run_1_NB.txt


In [157]:
print("Starting evaluations ----------->")

def evaluationScore(goldPath, predPath):
    with open(goldPath, 'r', encoding='utf-8') as fread:
        gold = fread.readlines()
    with open(predPath, 'r', encoding='utf-8') as fp:
        pred = fp.readlines()
    result = exact_match_metric.compute(predictions=pred, references=gold)
    return result

# print("Validation ended.. Calculating score -->")
# print(datetime.now())

print("Exact match score --->")
print(evaluationScore(GOLD_FILE_PATH, OUTPUT_FILE_PATH))

# EVALUATION 2 ----------->
def evaluationScore(goldPath, predPath):
    with open(goldPath, 'r', encoding='utf-8') as fread:
        gold = fread.readlines()
    with open(predPath, 'r', encoding='utf-8') as fp:
        pred = fp.readlines()
    
    result = 0
    ct=0
    for g,p in zip(gold, pred):
        # print("GOLD")
        # print(g)
        # print("PRED")
        # print(p)
        evalscore = evaluateComposition(g, p)
        # print(evalscore)
        result += evalscore
        # print(result)
        ct+=1
    # result = exact_match_metric.compute(predictions=pred, references=gold)
    return result/ct

def evaluateComposition(gold, predicted):
    
    gold = ast.literal_eval(gold)
    try:
        predicted = ast.literal_eval(predicted)
    except:
        # print("Cannot parse")
        # print(predicted)
        return 0.0
    gold_sets = [set(compound_list) for compound_list in gold]
    predicted_sets = [set(compound_list) for compound_list in predicted]
    
    # print("INSIDE EC")
    # print(gold_sets)
    # print(predicted_sets)
    # print("CLOSE")
    
    precision=0
    recall=0

    for predicted_set in predicted_sets:
        if(predicted_set in gold_sets):
            precision+=1
    
    for gold_set in gold_sets:
        if(gold_set in predicted_sets):
            recall+=1   
    
    precision /= len(predicted_sets)
    recall /= len(gold_sets)
    
    if(precision==0 and recall ==0):
        f1=0.0
    else: 
        f1 = (2*precision*recall)/(precision+recall)
    
    return round(f1,2)


print("Metric score --->")
print(evaluationScore(GOLD_FILE_PATH, OUTPUT_FILE_PATH))




Starting evaluations ----------->
Exact match score --->
{'exact_match': 0.0}
Metric score --->
0.36142857142857143


In [ ]:
# with open(OUTPUT_FILE_PATH, 'r', encoding='utf-8') as fp:
#     pred = fp.readlines()


# pred_extract_df = pd.DataFrame(pred, columns=['PredExtract'])
# extract_df['PredExtract'] = extract_df['PredExtract'].apply(lambda x: x.replace('\n',''))
# extract_df['PredExtract'] = extract_df['PredExtract'].apply(str)
# extract_df['PredExtract'] = extract_df['PredExtract'].apply(lambda x: ast.literal_eval(x))


# # final_df = pd.concat([X_val, pred_extract_df], axis=1)
# X_val_extracted = extract_df[extract_df['pred'] == '1']
# pred_combined = pd.concat([X_val, pred_extract_df], axis=1)
# X_val_remaining = extract_df[extract_df['pred'] == '0']

# final_df = pd.concat([X_val, pred_extract_df], axis=0)


# def final_evaluation(goldLst, predLst):
#     result = 0
#     ct=0
#     for g,p in zip(goldLst, predLst):
#         # print("GOLD")
#         # print(g)
#         # print("PRED")
#         # print(p)
#         evalscore = evaluateComposition(g, p)
#         # print(evalscore)
#         result += evalscore
#         # print(result)
#         ct+=1
#     # result = exact_match_metric.compute(predictions=pred, references=gold)
#     return result/ct

# print("FINAL RESULT --->>>")
# print(final_evaluation(final_df['composition'].tolist(), final_df['PredExtract'].tolist()))



In [ ]:
# analysis_eval = []
# def compute_PRF1(pred, gold):
#     pd.concat([X_val,pd.DataFrame(pred),pd.DataFrame(gold)])


# def evaluationScore(goldPath, predPath):
#     with open(goldPath, 'r', encoding='utf-8') as fread:
#         gold = fread.readlines()
#     with open(predPath, 'r', encoding='utf-8') as fp:
#         pred = fp.readlines()
    
#     # precision, recall, f1 = compute_PRF1(pred, gold)
#     result = exact_match_metric.compute(predictions=pred, references=gold)
#     return result

# print("Validation ended.. Calculating score -->")
# print(datetime.now())

# print("F1 match score --->")
# print(evaluationScore(GOLD_FILE_PATH, OUTPUT_FILE_PATH))
